Imports

In [1]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn

  Using cached pandas-2.2.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata (89 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.2 MB/s eta 0:00:0000:01
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-macosx_10_9_x86_64.whl (12.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 10.7 MB/s eta 0:00:0000:0100:01
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached matplotlib-3.10.1-cp311-cp311-macosx_10_12_x86_64.whl.metadata 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Data Loading

In [ ]:
warehouse_df = pd.read_csv('data/Cloud Warehouse Compersion Chart.csv')
expense_df = pd.read_csv('data/Expense IIGF.csv')
catalog_df = pd.read_csv('data/May-2022.csv')
pl_df = pd.read_csv('data/P  L March 2021.csv')
sales_df = pd.read_csv('data/Sale Report.csv')
int_sales_df = pd.read_csv('data/International sale Report.csv')
amzn_sales_df = pd.read_csv('data/Amazon Sale Report.csv')

Data Cleaning and Preprocessing

In [4]:
def clean_data(df):
    df.fillna(method='ffill', inplace=True)
    df.drop_duplicates(inplace=True)
    return df

for df in [warehouse_df, expense_df, catalog_df, pl_df, sales_df, int_sales_df]:
    clean_data(df)

/var/folders/nr/khgwm2yx29l9jhcvccmdjmn8q2_1zk/T/ipykernel_35277/1708858072.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [5]:
# Convert 'Sale_Date' to datetime, setting invalid dates to NaT
int_sales_df['Sale_Date'] = pd.to_datetime(int_sales_df['Sale_Date'], format='%m/%d/%Y', errors='coerce')

# Display the first few rows to verify the changes
print(int_sales_df['Sale_Date'].head())

0   2021-06-05
1   2021-06-05
2   2021-06-05
3   2021-06-05
4   2021-06-05
Name: Sale_Date, dtype: datetime64[ns]


In [6]:
# Format currency columns
currency_cols = ['Transfer_Price', 'Old_MRP', 'Final_Old_MRP', 'Ajio_MRP', 'Amazon_MRP', 'Amazon_FBA_MRP', 'Flipkart_MRP', 'Limeroad_MRP', 'Myntra_MRP', 'Paytm_MRP', 'Snapdeal_MRP']
for col in currency_cols:
    catalog_df[col] = catalog_df[col].astype(float)

# Feature Engineering
def calculate_profit_margin(df):
    df['Profit_Margin'] = (df['Final_Old_MRP'] - df['Transfer_Price']) / df['Final_Old_MRP'] * 100
    return df

catalog_df = calculate_profit_margin(catalog_df)

ValueError: could not convert string to float: '#VALUE!'

Exploratory Data Analysis

In [ ]:
def plot_sales_trend(df):
    plt.figure(figsize=(12,6))
    df.groupby('Sale_Date')['Gross_Amount'].sum().plot()
    plt.title('Daily Sales Trend')
    plt.xlabel('Date')
    plt.ylabel('Gross Amount')
    plt.show()

plot_sales_trend(int_sales_df)

def plot_top_products(df):
    top_products = df.groupby('Product_Style')['Quantity_Purchased'].sum().sort_values(descending=True).head(10)
    plt.figure(figsize=(10,6))
    top_products.plot(kind='bar')
    plt.title('Top 10 Selling Products')
    plt.xlabel('Product Style')
    plt.ylabel('Quantity Sold')
    plt.xticks(rotation=45)
    plt.show()

plot_top_products(sales_df)

def plot_price_distribution(df):
    price_cols = ['Ajio_MRP', 'Amazon_MRP', 'Flipkart_MRP', 'Myntra_MRP']
    plt.figure(figsize=(10,6))
    df[price_cols].plot(kind='box')
    plt.title('Price Distribution Across Platforms')
    plt.ylabel('Price')
    plt.show()

plot_price_distribution(catalog_df)

# Correlation Analysis
def plot_correlation_heatmap(df):
    corr_matrix = df.corr()
    plt.figure(figsize=(12,10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.title('Correlation Heatmap')
    plt.show()

plot_correlation_heatmap(catalog_df[currency_cols + ['Product_Weight', 'Profit_Margin']])

# Sales Channel Comparison
def compare_sales_channels(df):
    channel_sales = df.groupby('Product_Category')['Quantity_Purchased'].sum().sort_values(descending=True)
    plt.figure(figsize=(10,6))
    channel_sales.plot(kind='bar')
    plt.title('Sales by Product Category')
    plt.xlabel('Product Category')
    plt.ylabel('Quantity Sold')
    plt.xticks(rotation=45)
    plt.show()

compare_sales_channels(sales_df)

# Customer Insights
def analyze_customer_preferences(df):
    size_preference = df['Product_Size'].value_counts()
    plt.figure(figsize=(8,6))
    size_preference.plot(kind='pie', autopct='%1.1f%%')
    plt.title('Customer Size Preference')
    plt.ylabel('')
    plt.show()

analyze_customer_preferences(sales_df)

# Profitability Analysis
def analyze_profitability(df):
    plt.figure(figsize=(10,6))
    plt.scatter(df['Transfer_Price'], df['Profit_Margin'])
    plt.title('Transfer Price vs Profit Margin')
    plt.xlabel('Transfer Price')
    plt.ylabel('Profit Margin (%)')
    plt.show()

analyze_profitability(catalog_df)